In [2]:
from imutils.object_detection import non_max_suppression
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pytesseract

In [32]:
args={
      "image" : "Photos\image2.jpg",
      "east" : "frozen_east_text_detection.pb",
      "confidence" : 0.5,
      "width" : 320,
      "height" : 320,
      "padding" :0.0,
     }
pytesseract.pytesseract.tesseract_cmd = 'C:/Users/rishu/AppData/Local/Programs/Tesseract-OCR/tesseract.exe'

In [33]:
image = cv2.imread(args["image"])
img_box = pytesseract.image_to_data(image)
print(img_box)

level	page_num	block_num	par_num	line_num	word_num	left	top	width	height	conf	text
1	1	0	0	0	0	0	0	779	422	-1	
2	1	1	0	0	0	160	149	454	74	-1	
3	1	1	1	0	0	160	149	454	74	-1	
4	1	1	1	1	0	160	149	454	25	-1	
5	1	1	1	1	1	160	149	242	25	91.524475	INFORMATION
5	1	1	1	1	2	435	149	179	25	91.524475	RETRIEVAL
4	1	1	1	2	0	160	198	142	25	-1	
5	1	1	1	2	1	160	198	142	25	96.238647	PROJECT



In [34]:
orig = image.copy()
(origH, origW) = image.shape[:2]

(newW, newH) = (args["width"], args["height"])
rW = origW / float(newW)
rH = origH / float(newH)

image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]

layerNames = ["feature_fusion/Conv_7/Sigmoid",
              "feature_fusion/concat_3"]

net = cv2.dnn.readNet(args["east"])

blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
    (123.68, 116.78, 103.94), swapRB=True, crop=False)

net.setInput(blob)
(scores, geometry) = net.forward(layerNames)

(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []

for y in range(0, numRows):

    scoresData = scores[0, 0, y]
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]
    anglesData = geometry[0, 4, y]
    
    for x in range(0, numCols):

        if scoresData[x] < args["confidence"]:
            continue

        (offsetX, offsetY) = (4*x, 4*y)
        angle = anglesData[x]
        cos = np.cos(angle)
        sin = np.sin(angle)
        h = xData0[x] + xData2[x]
        w = xData1[x] + xData3[x]
        endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
        endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
        startX = int(endX - w)
        startY = int(endY - h)
        rects.append((startX, startY, endX, endY))
        confidences.append(scoresData[x])
        
boxes = non_max_suppression(np.array(rects), probs=confidences)
results = []

for (startX, startY, endX, endY) in boxes:
    startX = int(startX * rW)
    startY = int(startY * rH)
    endX = int(endX * rW)
    endY = int(endY * rH)
    
    dX = int((endX - startX) * args["padding"])
    dY = int((endY - startY) * args["padding"])
    
    startX = max(0, startX - dX)
    startY = max(0, startY - dY)
    endX = min(origW, endX + (dX * 2))
    endY = min(origH, endY + (dY * 2))

    r = orig[startY:endY, startX:endX]

    configuration = ("-l eng --oem 1 --psm 7")
    text = pytesseract.image_to_string(r, config=configuration)
    
    results.append(((startX, startY, endX, endY), text))
    
orig_image = orig.copy()

for ((start_X, start_Y, end_X, end_Y), text) in results:
    

    text = "".join([x if ord(x) < 128 else "" for x in text]).strip()
    cv2.rectangle(orig_image, (start_X, start_Y), (end_X, end_Y),
        (0, 0, 255), 1)
    cv2.putText(orig_image, text, (start_X, start_Y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.8,(255,0, 0), 2)

cv2.imshow("Text Detection", orig_image)
cv2.waitKey(0)

-1